In [21]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import regularizers, optimizers
from keras.utils import to_categorical

In [10]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


In [11]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
import cv2

# Image enhancements
def enhance_image(image):
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    return blurred

enhanced_x_train = np.array([enhance_image(image) for image in x_train])

# Enhance the images in x_test
enhanced_x_test = np.array([enhance_image(image) for image in x_test])


In [94]:
x_train.shape

(50000, 32, 32, 3)

In [95]:
enhanced_x_train.shape

(50000, 32, 32, 3)

In [97]:
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=40,  # rotate images randomly by 10 degrees
    shear_range=0.2,  # apply shear transformation with a shear angle of 20 degrees
    zoom_range=0.2,  # zoom in/out on images by 20%
    horizontal_flip=True,  # flip images horizontally
    fill_mode='nearest'  # fill any newly created pixels after rotation or shifting
)

# Generate augmented images from x_train
augmented_train = datagen.flow(enhanced_x_train, y_train, 128)

# Generate augmented images from x_test
augmented_test = datagen.flow(enhanced_x_test, y_test, 128)

In [76]:
augmented_train[0][0].shape

(64, 32, 32, 3)

In [107]:
model = Sequential()

model.add(Conv2D(32, (4,4), activation='relu', input_shape = (32,32,3)))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (4,4), activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(128, (4,4), activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation ='relu'))
model.add(Dense(10, activation ='softmax'))

In [108]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [109]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_56 (Conv2D)          (None, 29, 29, 32)        1568      
                                                                 
 max_pooling2d_49 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_57 (Conv2D)          (None, 11, 11, 64)        32832     
                                                                 
 max_pooling2d_50 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_58 (Conv2D)          (None, 2, 2, 128)         131200    
                                                                 
 max_pooling2d_51 (MaxPoolin  (None, 1, 1, 128)      

In [110]:
model.fit(enhanced_x_train, y_train, epochs=30, validation_data=(enhanced_x_test, y_test))

Epoch 1/30
1563/1563 [==============================] - 8s 5ms/step - loss: 1.7804 - accuracy: 0.3380 - val_loss: 1.4670 - val_accuracy: 0.4617
Epoch 2/30
1563/1563 [==============================] - 7s 4ms/step - loss: 1.5068 - accuracy: 0.4502 - val_loss: 1.3486 - val_accuracy: 0.5050
Epoch 3/30
1563/1563 [==============================] - 9s 6ms/step - loss: 1.4152 - accuracy: 0.4895 - val_loss: 1.3082 - val_accuracy: 0.5269
Epoch 4/30
1563/1563 [==============================] - 7s 4ms/step - loss: 1.3516 - accuracy: 0.5114 - val_loss: 1.2478 - val_accuracy: 0.5544
Epoch 5/30
1563/1563 [==============================] - 7s 4ms/step - loss: 1.3016 - accuracy: 0.5343 - val_loss: 1.2483 - val_accuracy: 0.5514
Epoch 6/30
1563/1563 [==============================] - 6s 4ms/step - loss: 1.2611 - accuracy: 0.5504 - val_loss: 1.2195 - val_accuracy: 0.5685
Epoch 7/30
1563/1563 [==============================] - 7s 4ms/step - loss: 1.2276 - accuracy: 0.5623 - val_loss: 1.2062 - val_accuracy:

In [111]:
model1.evaluate(enhanced_x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 1.3293 - accuracy: 0.5336


[1.329317569732666, 0.5335999727249146]

In [68]:
model1 = Sequential()
model1.add(Conv2D(32, (4,4), activation='relu', input_shape = (32,32,3)))
model1.add(MaxPooling2D(pool_size = (2,2)))

model1.add(Conv2D(64, (4,4), activation='relu'))
model1.add(MaxPooling2D(pool_size = (2,2)))

model1.add(Conv2D(128, (4,4), activation='relu'))
model1.add(MaxPooling2D(pool_size = (2,2)))

model1.add(Flatten())
model1.add(Dropout(0.5))
model1.add(Dense(128, activation ='relu'))
model1.add(Dense(10, activation ='softmax'))

In [69]:
model1.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [77]:
model1.fit(augmented_train, epochs=30, validation_data=augmented_test)

Epoch 1/30
782/782 [==============================] - 17s 21ms/step - loss: 1.8586 - accuracy: 0.3107 - val_loss: 1.5853 - val_accuracy: 0.4202
Epoch 2/30
782/782 [==============================] - 16s 20ms/step - loss: 1.5971 - accuracy: 0.4184 - val_loss: 1.4503 - val_accuracy: 0.4770
Epoch 3/30
782/782 [==============================] - 17s 21ms/step - loss: 1.5009 - accuracy: 0.4599 - val_loss: 1.3663 - val_accuracy: 0.5135
Epoch 4/30
782/782 [==============================] - 16s 20ms/step - loss: 1.4402 - accuracy: 0.4847 - val_loss: 1.3094 - val_accuracy: 0.5378
Epoch 5/30
782/782 [==============================] - 16s 20ms/step - loss: 1.3945 - accuracy: 0.5020 - val_loss: 1.2851 - val_accuracy: 0.5463
Epoch 6/30
782/782 [==============================] - 16s 20ms/step - loss: 1.3633 - accuracy: 0.5145 - val_loss: 1.2447 - val_accuracy: 0.5620
Epoch 7/30
782/782 [==============================] - 15s 20ms/step - loss: 1.3273 - accuracy: 0.5314 - val_loss: 1.2426 - val_accuracy:

In [78]:
model1.evaluate(augmented_test)

157/157 [==============================] - 3s 17ms/step - loss: 1.0994 - accuracy: 0.6301


[1.09935462474823, 0.6301000118255615]

In [79]:
model.evaluate(augmented_test)

157/157 [==============================] - 3s 16ms/step - loss: 1.1130 - accuracy: 0.6155


[1.1130120754241943, 0.6154999732971191]